In [1]:
import os
import torch
import shutil
from pathlib import Path
from tqdm.notebook import tqdm
from transformers import YolosFeatureExtractor, YolosForObjectDetection
from PIL import Image

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_doggo(src='raw_doggo', dst='cropped_doggos', conf=0.9, class_names = ['lhasa_apso', 'maltese', 'pekingese', 'shih_tzu']):
    """
    Detects and crops dog images from a given source directory using YOLO-based object detection.

    Args:
        src (str): Path to the source directory containing images of dog breeds.
                   Defaults to 'raw_doggo'.
        dst (str): Path to the destination directory where cropped dog images will be saved.
                   Defaults to 'cropped_doggos'.
        conf (float): Confidence threshold for dog detection. Only objects with confidence scores
                      above this threshold will be considered as dogs. Defaults to 0.9.
        class_names (list): List of class names for dog breeds to be detected and cropped.
                            Defaults to ['lhasa_apso', 'maltese', 'pekingese', 'shih_tzu'].

    Returns:
        None
    """
    confidence = conf

    # Specify the four class names
    class_names = class_names

    # Load the models outside the loop
    feature_extractor = YolosFeatureExtractor.from_pretrained('hustvl/yolos-tiny') # convolution layer
    model = YolosForObjectDetection.from_pretrained('hustvl/yolos-tiny') # detection layer, model object

    # Loop through each class folder
    for class_name in class_names:
        idx = 0
        class_folder = os.path.join(src, class_name)

        # Loop through each image inside the class folder
        for image_filename in tqdm(os.listdir(class_folder)):
            image_filepath = os.path.join(class_folder, image_filename)
            try:
                image = Image.open(image_filepath)
            except:
                pass

            inputs = feature_extractor(images=image, return_tensors="pt")
            outputs = model(**inputs)

            logits = outputs.logits # (num_images, num_boxes, num_classes)
            bboxes = outputs.pred_boxes # (num_images, num_boxes, coords)

            probs = torch.exp(logits[0]) / torch.exp(logits[0]).sum(axis=1).unsqueeze(1)

            target_sizes = torch.tensor([image.size[::-1]])
            results = feature_extractor.post_process(outputs, target_sizes=target_sizes)[0]

            # Identify objects in the image
            objects = {'labels': [], 'scores': [], 'boxes': []}

            for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                box = [round(i, 2) for i in box.tolist()]
                if score > confidence:
                    label_name = model.config.id2label[label.item()]
                    if label_name == 'dog':
                        print(
                            f"{idx} -- Detected {class_name} with confidence "
                            f"{round(score.item(), 3)} in {image_filename}"
                        )
                        objects['labels'].append(label_name)
                        objects['scores'].append(round(score.item(), 3))
                        objects['boxes'].append(box)

            # Create a folder for cropped images specific to the current class
            cropped_folder = os.path.join(dst, class_name)
            os.makedirs(cropped_folder, exist_ok=True)

            # Crop and save the detected objects
            for box in objects['boxes']:
                x_min, y_min, x_max, y_max = box
                cropped_image = image.crop((x_min, y_min, x_max, y_max))

                # Save the cropped image inside the class-specific cropped folder
                cropped_image = cropped_image.convert('RGB')
                cropped_image.save(os.path.join(cropped_folder, f"image_{idx}.jpg"))
                idx += 1

        print(f'\n\nThere are {len(os.listdir(cropped_folder))} doggo images in the {class_name} folder.\n')

In [3]:
def create_dataset(src, dst, range_, class_):
    """Copy images of class class_ within range_ from src to dst.
    
    Parameters
    ----------
    src : str
        source directory
    dst : str
        destination directory
    range_ : tuple
        tuple of min and max image index to copy
    class_ : str
        image class
    """
    if os.path.exists(dst):
        shutil.rmtree(dst)
    os.makedirs(dst)
    fnames = [f'image_{i}.jpg' for i in range(*range_)]
    for fname in fnames:
        src_file = os.path.join(src, fname)
        dst_file = os.path.join(dst, fname)
        shutil.copyfile(src_file, dst_file)

In [4]:
get_doggo()

/home/msds2023/clugod/.local/lib/python3.10/site-packages/transformers/models/yolos/feature_extraction_yolos.py:28: FutureWarning: The class YolosFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use YolosImageProcessor instead.
  warnings.warn(
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


  0%|          | 0/99 [00:00<?, ?it/s]

`post_process` is deprecated and will be removed in v5 of Transformers, please use `post_process_object_detection`


0 -- Detected lhasa_apso with confidence 1.0 in Lahsa-Apso_6.jpeg
1 -- Detected lhasa_apso with confidence 0.999 in download (3).jpeg
2 -- Detected lhasa_apso with confidence 0.999 in images (6).jpeg
3 -- Detected lhasa_apso with confidence 0.998 in images (1).jpeg
4 -- Detected lhasa_apso with confidence 0.998 in images (3).jpeg
5 -- Detected lhasa_apso with confidence 0.998 in images (7).jpeg
6 -- Detected lhasa_apso with confidence 0.987 in images (5).jpeg
7 -- Detected lhasa_apso with confidence 0.999 in AdobeStock_413918581-445x445.jpeg
8 -- Detected lhasa_apso with confidence 0.993 in nbc_dogshow_nonsporting_lhasaapso_191125.jpeg
8 -- Detected lhasa_apso with confidence 0.991 in nbc_dogshow_nonsporting_lhasaapso_191125.jpeg
10 -- Detected lhasa_apso with confidence 0.995 in 360_F_279010099_91d03gwp8qjpCXsDy2FHjLqmPXQM2xU2.jpeg
11 -- Detected lhasa_apso with confidence 0.998 in BREED Hero_0079_lhasa_apso.jpeg
12 -- Detected lhasa_apso with confidence 0.996 in file_23154_lhasa-apso

  0%|          | 0/94 [00:00<?, ?it/s]

0 -- Detected maltese with confidence 0.994 in images (2).jpeg
1 -- Detected maltese with confidence 0.997 in dog-breed-maltese-dog-grass.jpeg
2 -- Detected maltese with confidence 0.984 in images (4).jpeg
3 -- Detected maltese with confidence 0.984 in .DS_Store
4 -- Detected maltese with confidence 0.996 in gettyimages-171583654-612x612.jpeg
5 -- Detected maltese with confidence 0.999 in download (1).jpeg
6 -- Detected maltese with confidence 0.995 in Maltese-min-1.jpeg
7 -- Detected maltese with confidence 0.992 in download (4).jpeg
8 -- Detected maltese with confidence 0.978 in images (3).jpeg
9 -- Detected maltese with confidence 0.998 in maltese-2.jpeg
10 -- Detected maltese with confidence 0.997 in download (2).jpeg
11 -- Detected maltese with confidence 0.998 in images (5).jpeg
12 -- Detected maltese with confidence 0.999 in maltese-cover.jpeg
13 -- Detected maltese with confidence 1.0 in Teacup-Maltese-Feature-678x381.jpeg
14 -- Detected maltese with confidence 1.0 in dog-10377

  0%|          | 0/85 [00:00<?, ?it/s]

0 -- Detected pekingese with confidence 0.999 in pekingesef6.jpeg
1 -- Detected pekingese with confidence 0.999 in images (2).jpeg
2 -- Detected pekingese with confidence 0.99 in download.jpeg
3 -- Detected pekingese with confidence 0.995 in GettyImages-186699738-5eaf8e80458c4e19829874ec7352d06a.jpeg
4 -- Detected pekingese with confidence 0.999 in pekingese-dog-against-white-background.jpeg
5 -- Detected pekingese with confidence 0.994 in shutterstock_627372341-1.jpeg
6 -- Detected pekingese with confidence 0.997 in download (3).jpeg
7 -- Detected pekingese with confidence 0.997 in images (4).jpeg
8 -- Detected pekingese with confidence 0.997 in .DS_Store
9 -- Detected pekingese with confidence 0.998 in images.jpeg
10 -- Detected pekingese with confidence 0.944 in Bailey_Pekingese.jpeg
10 -- Detected pekingese with confidence 0.931 in Bailey_Pekingese.jpeg
10 -- Detected pekingese with confidence 0.914 in Bailey_Pekingese.jpeg
13 -- Detected pekingese with confidence 1.0 in 2560px-Pek

  0%|          | 0/80 [00:00<?, ?it/s]

0 -- Detected shih_tzu with confidence 0.911 in images (2).jpeg
1 -- Detected shih_tzu with confidence 0.984 in download.jpeg
2 -- Detected shih_tzu with confidence 0.98 in Dog-Shih_Tzu-A_beautiful_little_Shih_Tzu_standing_tall,_showing_off_its_big_bushy_tail.jpeg
3 -- Detected shih_tzu with confidence 0.998 in dog-g6ba04626c_1280.jpeg
4 -- Detected shih_tzu with confidence 0.99 in xEye_Problem1.jpg.pagespeed.ic.5ZSgsWoVo-.jpg
5 -- Detected shih_tzu with confidence 0.994 in istockphoto-466754640-612x612.jpeg
6 -- Detected shih_tzu with confidence 0.994 in .DS_Store
7 -- Detected shih_tzu with confidence 0.958 in images.jpeg
8 -- Detected shih_tzu with confidence 0.986 in gettyimages-1170243723-612x612.jpeg
9 -- Detected shih_tzu with confidence 0.967 in download (1).jpeg
10 -- Detected shih_tzu with confidence 0.922 in images (1).jpeg
11 -- Detected shih_tzu with confidence 0.916 in images (3).jpeg
12 -- Detected shih_tzu with confidence 1.0 in breed-profile-shih-tzu-991830_2000x.jpeg


In [5]:
src = 'cropped_doggos'
class_names = ['lhasa_apso', 'maltese', 'pekingese', 'shih_tzu']

# looping through create_dataset for each class
class_counts = [len(os.listdir(os.path.join(src, class_))) for class_ in class_names]  # Number of pictures for each class

for class_, count in zip(class_names, class_counts):

    # Define the custom ranges for each split based on the available count
    train_range = (1, int(0.7 * count)) # 70% for training
    validation_range = (int(0.7 * count) + 1, int(0.9 * count)) # 20% for validation
    test_range = (int(0.9 * count) + 1, count) # 10% for testing
    
    dst = os.path.join(src, 'train', class_)
    create_dataset(os.path.join(src, class_), dst, range_=train_range, class_=class_)

    dst = os.path.join(src, 'validation', class_)
    create_dataset(src+'/'+class_, dst, range_=validation_range, class_=class_)

    dst = os.path.join(src, 'test', class_)
    create_dataset(src+'/'+class_, dst, range_=test_range, class_=class_)